## Exploration of DeepLift with movie reviews

**Function        : Exploration of DeepLift with movie reviews**<br>
**Author          : Team DIANNA**<br>
**Contributor     :**<br>
**First Built     : 2021.10.26**<br>
**Last Update     : 2021.10.26**<br>
**Library         : os, numpy, matplotlib, torch, captum**<br>
**Description     : In this notebook we test XAI method DeepLift using trained movie reviews model.**<br>
**Return Values   : Sliency maps**<br>
**Note**          : We use Captum library to perform DeepLift. This library works only with pytorch and it is not compitable with onnx.<br>


In [1]:
%matplotlib inline
from captum.attr import DeepLift
import numpy as np
# onnx module
import onnxruntime as ort
# text data processing
import spacy
from scipy.special import expit
from torchtext.data import get_tokenizer
from torchtext.vocab import Vectors
# pytorch
import torch
from torch import nn, optim
import torch.nn.functional as F

In [9]:
base_dir = '/Users/pbos/SURFdrive/Shared/datasets/stanford_sentiment_treebank_v2/'
dianna_base_dir = '/Users/pbos/projects/dianna/dianna/'

# please specify model path
onnx_model_path = base_dir + 'movie_review_model.onnx'
torch_model_path = base_dir + 'movie_review_model.pytorch'
# word vector
word_vector_file = dianna_base_dir + 'tests/test_data/word_vectors.txt'

In [10]:
class ModelRunner():
    def __init__(self, model_path, word_vector_file, max_filter_size):
        self.filename = model_path
        # ensure the spacy english is downloaded
        spacy.cli.download('en_core_web_sm')
        self.tokenizer = get_tokenizer('spacy', 'en_core_web_sm')
        self.vocab = Vectors(word_vector_file, cache='.')

        self.max_filter_size = max_filter_size

    def __call__(self, sentences):
        output = []

        sess = ort.InferenceSession(self.filename)
        input_name = sess.get_inputs()[0].name
        output_name = sess.get_outputs()[0].name

        for sentence in self.normalize_sentences(sentences):
            # feed to model
            onnx_input = {input_name: [sentence]}
            pred = expit(sess.run([output_name], onnx_input)[0])

            # output 2 classes
            positivity = pred[:, 0]
            negativity = 1 - positivity

            output.append(np.transpose([negativity, positivity])[0])

        return np.array(output)

    def normalize_sentences(self, sentences):
        """
        Convert input to desired format.
        """
        if isinstance(sentences, str):
            return [self.numericalizer(sentences)]
        elif isinstance(sentences, list):
            if isinstance(sentences[0], str):
                return [self.numericalizer(sentence) for sentence in sentences]
            elif torch.is_tensor(sentences[0]):
                return sentences
    
    def numericalizer(self, sentence):
        """
        From sentence to tensor.
        """
        # get tokens
        tokens = self.tokenizer(sentence)
        if len(tokens) < self.max_filter_size:
            tokens += ['<pad>'] * (self.max_filter_size - len(tokens))

        # numericalize
        tokens = [self.vocab.stoi[token] if token in self.vocab.stoi else self.vocab.stoi['<unk>'] for token in
                  tokens]

        # add required batch axis
        tokens = torch.tensor(tokens).unsqueeze(0)

        return torch.as_tensor(tokens)#.requires_grad_()#.type(torch.FloatTensor)


In [6]:
runner = ModelRunner(onnx_model_path, word_vector_file, max_filter_size=5)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


100%|█████████▉| 13888/13889 [00:00<00:00, 31929.01it/s]


In [7]:
# create a model
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_size, n_filters, filter_sizes, padding_idx,
                dropout, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)
        
        self.conv_layers = nn.ModuleList()
        for filter_size in filter_sizes:
            layer = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_size, embedding_size))
            self.conv_layers.append(layer)

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters * len(filter_sizes), output_dim)
    
    def forward(self, text):
        # shape = [batch size, max nword per sentence]
        embedding = self.embedding(text).unsqueeze(1)
        # shape = [batch_size, 1, nword, embedding dim]
        conved = [F.relu(conv(embedding)).squeeze(3) for conv in self.conv_layers]
        # shape = len(filter_sizes) list of [batch_size, n_filter, nword - filter_size + 1]
        # note: max_pool1d does not work with ONNX when output shape is dynamic
        # therefore switched to adaptive_max_pool1d
        pooled = [F.adaptive_max_pool1d(out, 1).squeeze(2) for out in conved]
        # shape = len(filter_sizes) list of [batch_size, n_filter]
        concat = torch.cat(pooled, dim=1)
        # shape = [batch_size * len(filter_sizes), n_filter]
        dropped = self.dropout(concat)
        return self.fc(dropped)

In [58]:
class SimpleTestModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, padding_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)

    def forward(self, text):
        # shape = [batch size, max nword per sentence]
        print(text)
        embedding = self.embedding(text).unsqueeze(1)
        flat = torch.flatten(embedding)
        return flat

In [59]:
simple_model = SimpleTestModel(13889, 100, 1)

In [60]:
# load best model from disk
model = torch.load(torch_model_path, map_location=torch.device('cpu'))
#model = loaded_model.to(device)
model.eval()

Model(
  (embedding): Embedding(13889, 100, padding_idx=1)
  (conv_layers): ModuleList(
    (0): Conv2d(1, 245, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 245, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 245, kernel_size=(5, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.6913344449168243, inplace=False)
  (fc): Linear(in_features=735, out_features=1, bias=True)
)

In [18]:
review = 'such a bad movie'

In [36]:
explain_this = runner.numericalizer(review)

In [61]:
explain_this

tensor([[129,   4,  73,  21,   1]])

In [62]:
deeplift_simple = DeepLift(simple_model)
attributions_dl_simple = deeplift_simple.attribute(explain_this)

tensor([[129,   4,  73,  21,   1],
        [  0,   0,   0,   0,   0]])


RuntimeError: One of the differentiated Tensors does not require grad

In [63]:
#deeplift = DeepLift(runner)
deeplift = DeepLift(model)
attributions_dl = deeplift.attribute(explain_this)

RuntimeError: One of the differentiated Tensors does not require grad